In [ ]:
pip install ollama

In [ ]:
from langchain.llms import Ollama

In [ ]:
ollama = Ollama (base_url= "http://localhost:11434",model="sql_gen_model")

In [ ]:
print(ollama("who are you"))

In [ ]:
import pandas as pd

#csv file is loaded
csv_file_path = './dummycsv/student-dataset.csv'
df = pd.read_csv(csv_file_path)

#column names and data type is given
columns = df.columns
data_types = df.dtypes


#mapping data types to sql query data types
data_type_mapping = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'TEXT',  # assuming all non-numeric data is text
    'datetime64[ns]': 'DATETIME'
}

#constructing the CREATE TABLE query
table_name = ''
create_table_query = f"CREATE TABLE {table_name} (\n"
for col in columns:
    sql_data_type = data_type_mapping[str(data_types[col])]
    create_table_query += f"    {col} {sql_data_type},\n"
create_table_query = create_table_query.rstrip(',\n') + "\n);"

print(create_table_query)



#generate the INSERT INTO statements
insert_queries = []
for index, row in df.iterrows():
    values = ', '.join([f"'{str(value)}'" if pd.notnull(value) else 'NULL' for value in row.values])
    insert_query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({values});"
    insert_queries.append(insert_query)

#printing the first few INSERT statements for verification
for query in insert_queries[:]:
    print(query)

In [ ]:
import pandas as pd
import subprocess

#csv file is loaded
csv_file_path = './dummycsv/student-dataset.csv'  #replacing the csv file...
df = pd.read_csv(csv_file_path)


csv_preview = df.head().to_string(index=False)

#asking the user to input what query to be generated
user_question = input("What SQL query do you want based on the CSV data? ").strip()


prompt = f"Here is a sample of the CSV data:\n\n{csv_preview}\n\nBased on this data, generate a SQL query that answers the following question: {user_question}"


def query_llm(prompt):
    try:
        #Prepare and run the subprocess to interact with the local LLM
        process = subprocess.Popen(
            ['ollama', 'run', 'sql_gen_model'],  
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        #prompt is sent to llm
        stdout, stderr = process.communicate(prompt)
        
        if process.returncode == 0:
            return stdout.strip()
        else:
            return f"Error: {stderr.strip()}"
    except Exception as e:
        return f"An exception occurred: {str(e)}"

#prompt is sent to llm
response = query_llm(prompt)

#o/p the LLM's generated SQL query
print("\nGenerated SQL Query:")
print(response)

In [ ]:
pip install streamlit

In [ ]:
import streamlit as st
import pandas as pd
import subprocess

# Function to interact with the local LLM
def query_llm(prompt):
    try:
        # Prepare and run the subprocess to interact with the local LLM
        process = subprocess.Popen(
            ['ollama', 'run', 'sql_gen_model'],  
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        # Prompt is sent to LLM
        stdout, stderr = process.communicate(prompt)
        
        if process.returncode == 0:
            return stdout.strip()
        else:
            return f"Error: {stderr.strip()}"
    except Exception as e:
        return f"An exception occurred: {str(e)}"

# Set up the page configuration
st.set_page_config(page_title="Querify", layout="centered")

# Title
st.title("Querify")

# CSV file upload
uploaded_file = st.file_uploader("Upload your CSV file", type=["csv"])

if uploaded_file is not None:
    # Read the CSV file
    df = pd.read_csv(uploaded_file , encoding='utf-8')
    st.write("Preview of uploaded CSV:")
    st.dataframe(df)

    

    # Input box for user question
    user_question = st.text_input("What SQL query do you want based on the CSV data?", placeholder="e.g., Show me all students with a grade higher than 90")

    # Button to generate the SQL query
    if st.button("Generate SQL Query"):
        # Prepare the prompt for LLM
        csv_preview = df.head().to_string(index=False)
        prompt = f"Here is a sample of the CSV data:\n\n{csv_preview}\n\nBased on this data, generate a SQL query that answers the following question: {user_question}"
        
        # Get the response from the LLM
        response = query_llm(prompt)
        
        # Display the generated SQL query
        st.markdown("### Generated SQL Query:")
        st.code(response)

        # Optional: Explain the generated SQL query if needed (this can be another call to LLM)
        explanation_prompt = f"Explain the following SQL query:\n{response}"
        explanation = query_llm(explanation_prompt)
        
        # Display the explanation
        st.markdown("### Explanation of the SQL Query:")
        st.code(explanation)

# Footer
st.markdown("<p style='text-align: center;'>Querify: SQL Query Generator</p>", unsafe_allow_html=True)
